In [ ]:
import openai
from dotenv import load_dotenv
import os

load_dotenv('../.env')

openai.api_key = os.getenv('OPENAI_API_KEY')
AL_API_KEY = os.getenv('AL_API_KEY')
AL_USER = os.getenv('AL_USER')

In [ ]:
from assemblyline_client import get_client
al_client = get_client("https://malware.cyber.gc.ca:443", apikey=(AL_USER, AL_API_KEY))

In [ ]:
import json

def get_antivirus_service_results_from_al(sha256):
    """Get the AssemblyLine antivirus report for the given sha256"""
    search_result = al_client.search.result(f'sha256:{sha256} AND response.service_name:AntiVirus', fl="*")

    if len(search_result['items']) > 0:
        return json.dumps([i['title_text'] for i in search_result['items'][0]['result']['sections']])
    else:
        return "No antivirus report available for this sha256 in AssemblyLine"
    
def run_conversation(prompt):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": prompt}]
    functions = [
        {
            "name": "get_antivirus_service_results_from_al",
            "description": "Get the AssemblyLine antivirus results for a given sha256",
            "parameters": {
                "type": "object",
                "properties": {
                    "sha256": {
                        "type": "string",
                        "description": "The sha256 value",
                    }
                },
                "required": ["sha256"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_antivirus_service_results_from_al": get_antivirus_service_results_from_al,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            sha256=function_args.get("sha256")
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

In [ ]:
response = run_conversation("What are the AssemblyLine antivirus results for sha256 af42459d9989ef3ff60d43e8069b647bfa2307e847452d2848dcc6f970984104?")

In [ ]:
print(response.choices[0].message.content)

In [ ]:
response = run_conversation("What are the AssemblyLine antivirus results for sha256 6db809dc08b230ed5878b049e2787369fd47fd4dd28aad3fd07dbd6a2408f8aa?")

In [ ]:
print(response.choices[0].message.content)